# Retrieve Route Attributes from Dijkstra
We have a list of edges and turns from Dijkstra, and we need to get the route attributes (how many turns, how much feet up, miles of bike facility, etc).

There are two different ones that we are looking for link attributes + turn attributes.

For link attributes we the linkid and direction of travel (because of elevation).
For turn attributes we just need linkid to linkid.

In [ ]:
from pathlib import Path
import time
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx

import modeling_turns
#import stochastic_optimization

In [ ]:
#fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot'
fp = Path.home() / 'Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/BikewaySim/Data'

with (fp / 'impedance_calibration.pkl').open('rb') as fh:
    (df_edges,pseudo_df,pseudo_G) = pickle.load(fh)

# Retrieve link/turn costs
default below is link distance

In [ ]:
link_costs = dict(zip(list(zip(df_edges['source'],df_edges['target'],df_edges['linkid'])),df_edges['length_ft']))
tup = list(zip(pseudo_df['source_A'],pseudo_df['source_B'],pseudo_df['source_linkid']))
pseudo_df['source_cost'] = list(map(link_costs.get,tup))
tup = list(zip(pseudo_df['target_A'],pseudo_df['target_B'],pseudo_df['target_linkid']))

pseudo_df['target_cost'] = list(map(link_costs.get,tup))
pseudo_df['total_cost'] = pseudo_df['source_cost'] + pseudo_df['target_cost'] #+turn_cost

costs = pseudo_df.groupby(['source','target'])['total_cost'].min()
nx.set_edge_attributes(pseudo_G,values=costs,name='weight')

In [ ]:
source = list(pseudo_G.nodes())[0]
target = list(pseudo_G.nodes())[420]
print(source,target)

In [ ]:
import networkx as nx
length, edge_list = nx.single_source_dijkstra(pseudo_G,source,target,weight="weight")
turn_list = [[edge_list[i][0],edge_list[i][1],edge_list[i+1][0],edge_list[i+1][1]] for i in range(len(edge_list)-1)]


In [ ]:
turn_cols = ['turn_type','signalized_left_straight','unsignalized_left_straight_nonlocal']
linkid_cols = ['source_linkid','source_reverse_link','target_linkid','target_reverse_link']
chosen_turns = pseudo_df.set_index(['source_A','source_B','target_A','target_B'],drop=False).loc[turn_list,linkid_cols+turn_cols]

tripid = 302

#make a single row dataframe to attach to trips_df
stats_dict = {}
stats_dict[tripid] = {
    'tripid':tripid,
    'signalized_left_straight': chosen_turns['signalized_left_straight'].sum(),
    'unsignalized_left_straight_nonlocal': chosen_turns['unsignalized_left_straight_nonlocal'].sum()
}
turn_dict = chosen_turns['turn_type'].value_counts().to_dict()
stats_dict[tripid].update(turn_dict)

# Case 1: Need to run shortest paths to create link sequence

In [ ]:
source_links = chosen_turns[['source_linkid','source_reverse_link']]
target_links = chosen_turns[['target_linkid','target_reverse_link']]
source_links.columns = ['linkid','reverse_link']
target_links.columns = ['linkid','reverse_link']
linkids = pd.concat([source_links,target_links],ignore_index=True).drop_duplicates()
chosen_links = df_edges.merge(linkids,on=['linkid','reverse_link'])

# Case 2: Already have link sequence

In [36]:
with (fp / '200_sample.pkl').open('rb') as fh:
    matched_traces = pickle.load(fh)

In [37]:
matched_traces.keys()

dict_keys([30724, 1545, 25098, 24587, 6668, 3599, 8207, 1041, 16401, 28696, 4633, 14366, 5663, 13857, 33826, 6179, 34340, 15397, 24614, 29230, 10286, 3630, 17460, 30264, 14909, 26176, 8258, 25155, 14406, 15434, 4684, 8780, 9806, 24146, 26198, 7255, 16473, 7257, 13919, 24162, 25186, 18531, 8802, 9320, 30825, 13418, 25192, 2156, 30830, 11377, 31353, 4220, 7293, 13960, 29837, 5265, 32406, 29335, 10903, 25755, 1182, 28830, 161, 33442, 9890, 30884, 27815, 2217, 28330, 1707, 30382, 3247, 8880, 693, 1208, 10937, 10427, 12995, 196, 29891, 9417, 3283, 24791, 4312, 4831, 7392, 737, 15584, 28896, 28900, 33503, 11492, 14049, 29928, 13543, 10474, 27371, 2284, 8429, 751, 10993, 17137, 8436, 18168, 5880, 25850, 13051, 28923, 7933, 4353, 24322, 29955, 11521, 32526, 7951, 18192, 1299, 26899, 24857, 16666, 1818, 27935, 28962, 27939, 17187, 28970, 10236, 25902, 16687, 29488, 6449, 32049, 30000, 308, 7477, 31549, 31039, 4416, 30016, 25417, 26443, 10575, 9556, 16213, 16214, 8539, 11102, 7013, 17766, 12647,

In [38]:
tripid = 24587
edges = matched_traces[tripid]['edges']
edges['reverse_link'] = edges['forward'] == False
list_of_edges = list(zip(edges['linkid'],edges['reverse_link']))
list_of_turns = [(list_of_edges[i][0],list_of_edges[i][1],list_of_edges[i+1][0],list_of_edges[i+1][1]) for i in range(0,len(list_of_edges)-1)]

In [40]:
list_of_turns

[(10976.0, False, 10976.0, True),
 (10976.0, True, 10976.0, False),
 (10976.0, False, 10976.0, True),
 (10976.0, True, 10976.0, False),
 (10976.0, False, 10976.0, True),
 (10976.0, True, 10979.0, True),
 (10979.0, True, 10972.0, False),
 (10972.0, False, 10973.0, False),
 (10973.0, False, 10978.0, False),
 (10978.0, False, 10977.0, True),
 (10977.0, True, 8149.0, False),
 (8149.0, False, 8134.0, True),
 (8134.0, True, 8157.0, False),
 (8157.0, False, 8157.0, True),
 (8157.0, True, 8135.0, False),
 (8135.0, False, 8151.0, False),
 (8151.0, False, 8146.0, False),
 (8146.0, False, 8147.0, False),
 (8147.0, False, 8148.0, False),
 (8148.0, False, 8139.0, False),
 (8139.0, False, 8138.0, False),
 (8138.0, False, 17088.0, False),
 (17088.0, False, 17088.0, True),
 (17088.0, True, 8140.0, False),
 (8140.0, False, 8152.0, True),
 (8152.0, True, 17106.0, True),
 (17106.0, True, 17106.0, False),
 (17106.0, False, 17106.0, True),
 (17106.0, True, 17106.0, False),
 (17106.0, False, 17106.0, True),

In [42]:
test = (5.0,False,0.0,True)
pseudo_df.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link']).loc[test]


source_A                                           783782570
source_B                                            67358015
target_A                                            67358015
target_B                                            67358019
source_azimuth                                         270.4
target_azimuth                                         178.5
azimuth_change                                         268.1
turn_type                                               left
source                                 (783782570, 67358015)
target                                  (67358015, 67358019)
signalized_left_straight                               False
unsignalized_left_straight_nonlocal                    False
source_cost                                        467.45937
target_cost                                       355.571011
Name: (5.0, False, 0.0, True), dtype: object

In [39]:
chosen_links = df_edges.set_index(['linkid','reverse_link']).loc[list_of_edges]
chosen_turns = pseudo_df.set_index(['source_linkid','source_reverse_link','target_linkid','target_reverse_link']).loc[list_of_turns]

KeyError: '[(10976.0, False, 10976.0, True), (10976.0, True, 10976.0, False), (8157.0, False, 8157.0, True), (17088.0, False, 17088.0, True), (17106.0, True, 17106.0, False), (17106.0, False, 17106.0, True), (17093.0, True, 17093.0, False), (17104.0, False, 17104.0, True), (17104.0, True, 17104.0, False), (8176.0, True, 8176.0, False), (8176.0, False, 8176.0, True), (18439.0, True, 18439.0, False), (18439.0, False, 18439.0, True), (8158.0, True, 8158.0, False), (8158.0, False, 8158.0, True)] not in index'

# Get Link Attributes

In [ ]:
summary_attributes = {}

#specify columns to summarize
bool_cols = ['bl','mu','pbl']
cols = ['link_type','highway','bridge','tunnel','speedlimit_range_mph','lanes_per_direction']

for bool_col in bool_cols:
    total_length = chosen_links[chosen_links[bool_col] == 1]['length_ft'].sum()
    summary_attributes[bool_col] = np.round(total_length/chosen_links['length_ft'].sum(),2)

for col in cols:
    for unique_val in chosen_links[col].unique():
        if (unique_val != None) | (unique_val == np.nan):
            total_length = chosen_links[chosen_links[col] == unique_val]['length_ft'].sum()
        else:
            continue
        if isinstance(unique_val,str) == False:
            unique_val = str(unique_val)
        summary_attributes[col+'.'+unique_val] = np.round(total_length/chosen_links['length_ft'].sum(),2)

stats_dict[tripid].update(summary_attributes)


In [ ]:
stats_dict

# Turn Attributes

In [ ]:
turn_cols = ['turn_type','signalized_left_straight','unsignalized_left_straight_nonlocal']
linkid_cols = ['source_linkid','source_reverse_link','target_linkid','target_reverse_link']
chosen_turns = pseudo_df.set_index(['source_A','source_B','target_A','target_B'],drop=False).loc[turn_list,linkid_cols+turn_cols]

tripid = 302

#make a single row dataframe to attach to trips_df
stats_dict = {}
stats_dict[tripid] = {
    'tripid':tripid,
    'signalized_left_straight': chosen_turns['signalized_left_straight'].sum(),
    'unsignalized_left_straight_nonlocal': chosen_turns['unsignalized_left_straight_nonlocal'].sum()
}
turn_dict = chosen_turns['turn_type'].value_counts().to_dict()
stats_dict[tripid].update(turn_dict)

In [ ]:
#TODO recalculate bearing
#create pseudo graph for modeling turns
df_edges, pseudo_links, pseudo_G = modeling_turns.create_pseudo_dual_graph(links,'A','B','linkid','oneway')


In [ ]:
source = list(pseudo_G.nodes())[0]
target = list(pseudo_G.nodes())[420]

In [ ]:
source = (68209677, 68209675)
target = (69200243, 69465418)

import networkx as nx
length, path = nx.single_source_dijkstra(pseudo_G,source,target)

In [ ]:
edge_list = [(linkids[i],linkids[i+1]) for i in range(len(linkids)-1)]
edge_list

In [ ]:
edge_list = [(*path[i],*path[i+1]) for i in range(len(path)-1)]
edge_list

In [ ]:
pseudo_links.columns

In [ ]:
pseudo_links.set_index(['source_A','source_B','target_A','target_B']).loc[edge_list,'turn_type'].value_counts()